In [19]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.impute import SimpleImputer
import numpy as np
from scipy.stats import pearsonr

In [3]:
meta = pd.read_csv('../data/E0022_P06_final_sample_map_no_control.txt', sep='\t')

ic50 = pd.read_csv('../data/drug/ic50_processed.csv')

data_sample = pd.read_csv('../data/protein/E0022_P06_Protein_Matrix_ProNorM_no_control.txt', sep='\t')

In [4]:
min_cell_lines = 300
ic50_counts = ic50.groupby(['Drug Id']).size()
selected_drugs = ic50_counts[ic50_counts > min_cell_lines].index.values

ic50_selected = ic50[ic50['Drug Id'].isin(selected_drugs)]

cell_lines_train = pd.read_csv(
    '../data/protein/E0022_P06_Protein_Matrix_ProNorM_no_control_train.txt',
    sep='\t')['Cell_line'].values
cell_lines_test = pd.read_csv(
    '../data/protein/E0022_P06_Protein_Matrix_ProNorM_no_control_test.csv',
    sep='\t')['Cell_line'].values

ic50_selected_train = ic50_selected[ic50_selected['Cell line name'].isin(
    cell_lines_train)]
ic50_selected_test = ic50_selected[ic50_selected['Cell line name'].isin(
    cell_lines_test)]

In [24]:
drug_id = 1909
target = 'IC50_norm_scaled'

In [25]:
tmp_df_train = pd.merge(
    data_sample,
    ic50_selected_train[ic50_selected_train['Drug Id'] == drug_id][[
        'Cell line name', target
    ]],
    how='inner',
    left_on='Cell_line',
    right_on='Cell line name')

tmp_df_test = pd.merge(
    data_sample,
    ic50_selected_test[ic50_selected_test['Drug Id'] == drug_id][[
        'Cell line name', target
    ]],
    how='inner',
    left_on='Cell_line',
    right_on='Cell line name')

In [26]:
X_train = tmp_df_train.drop(['Cell_line', 'Cell line name', target], axis=1)
X_test = tmp_df_test.drop(['Cell_line', 'Cell line name', target], axis=1)

y_train = tmp_df_train[target]
y_test = tmp_df_test[target]

In [27]:
imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-2.242616)

In [28]:
reg = DecisionTreeRegressor()

In [29]:
reg.fit(imputer.fit_transform(X_train), y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [31]:
reg.get_depth()

21

In [32]:
pearsonr(reg.predict(imputer.transform(X_test)), y_test)

(0.4677326217349733, 3.864377170012069e-09)